# For preprocessing training_data.pt
Jupyter notebook seemed easier to visualize the preprocessing

In [19]:
import sys
import os
import copy

import numpy as np
import pandas as pd

import pickle

from buffer import Buffer

In [20]:
pf = open('training_data.pt', 'rb')
buf = pickle.load(pf)
pf.close()

In [21]:
'''
OBS: {'outdoor_temp': 7.7, 'indoor_temp_living': 22.20591673389583, 'sky_diffuse_solar_ldf': 0.0, 
'sky_diffuse_solar_sdr': 0.0, 'site_direct_solar': 0.0, 'site_horizontal_infrared': 327.33333333333337, 
'elec_cooling': 0.0, 'hour': 4, 'day_of_week': 2, 'day': 2, 'month': 6, 'cost_rate': 2.4, 'cost': 0.0}
'''
OUTDOOR_TEMP = 0
INDOOR_TEMP = 1
DIFFUSE_SOLAR_LDF = 2
DIFFUSE_SOLAR_SDR = 3
SITE_DIRECT_SOLAR = 4
SITE_HORZ_INFRARED = 5
ELEC_COOLING = 6
HOUR = 7
DAY_OF_WEEK = 8
DAY = 9
MONTH = 10
COST_RATE = 11
COST = 12

buf.buffer[100]

(array([ 13.9       ,  20.37567474,  65.74163903,  65.09133458,
         58.66666667, 339.66666667,   0.        ,  16.        ,
          1.        ,   1.        ,   6.        ,   7.4       ,
          0.        ]),
 28.1,
 array([ 13.9       ,  20.37225053,  56.66793196,  56.10546973,
         44.        , 344.        ,   0.        ,  16.        ,
          1.        ,   1.        ,   6.        ,   7.4       ,
          0.        ]))

In [22]:
buf_copy = copy.deepcopy(buf)

In [23]:
    def _add_10_minutes(inp):
        year, month, day, hour, minute = inp

        # Calculate the total number of minutes
        total_minutes = (hour * 60) + minute + 10

        # Calculate the new hour and minute values
        new_hour = total_minutes // 60
        new_minute = total_minutes % 60

        # Handle hour and day overflow
        if new_hour >= 24:
            new_hour %= 24
            day += 1

        # Handle month and year overflow
        if month in [1, 3, 5, 7, 8, 10, 12] and day > 31:
            day = 1
            month += 1
        elif month in [4, 6, 9, 11] and day > 30:
            day = 1
            month += 1
        elif month == 2:
            if (year % 4 == 0 and year % 100 != 0) or year % 400 == 0:
                if day > 29:
                    day = 1
                    month += 1
            else:
                if day > 28:
                    day = 1
                    month += 1

        # Handle minute overflow and represent 0 minutes as 60
        if new_minute == 0:
            new_minute = 60
            new_hour -= 1

        if new_hour == -1:
            new_hour = 23

        return (year, month, day, new_hour, new_minute)
    
    def _get_cost_signal(day_of_week, hour, minute):
        '''get cost signal at given time. @param: minute is not used'''
        if day_of_week in [1, 7]:
            # weekend pricing
            if hour in range(0, 7) or hour in range(23, 24 + 1): # plus one is to include 7
                #self.next_obs['cost_rate'] = 2.4
                return 2.4
            elif hour in range(7, 23):
                #self.next_obs['cost_rate'] = 7.4
                return 7.4
        else:
            if hour in range(0, 7) or hour in range(23, 24 + 1):
                #self.next_obs['cost_rate'] = 2.4
                return 2.4
            elif hour in range(7, 16) or hour in range(21, 23):
                #self.next_obs['cost_rate'] = 10.2
                return 10.2
            elif hour in range(16, 21):
                #self.next_obs['cost_rate'] = 24.0
                return 24.0

In [27]:
'''
Delete:
cost_rate
cost

s_{t+1} = f(s_t, a_t)
'''
print('BUF LEN:', len(buf_copy.buffer))
for i in range(len(buf_copy.buffer)):
    data = buf_copy.buffer[i]
    s = data[0]
    s_p = data[2]
    np.delete(s, COST_RATE)
    np.delete(s, COST)
    np.delete(s_p, COST_RATE)
    np.delete(s_p, COST)

BUF LEN: 794880
